In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


Prepocess and Merge input Alabama Data.

**8. Choosing representative survey questions**
In order to get a good picture of tobacco use among each group, I chose some representative questions based on files the Alabama YTS 2016-MS codebook and 2016 YTS Questionnaire-final.

Cigaretter:

CR7: Have you ever tried cigarette smoking, even one or two puffs?
  1. Yes.
  2. No

CR11: How old were you when you first tried cigarette smoking, even one or two puffs?
  1. I have never smoked cigarettes, not even one or two puffs.
  2. 8 years old or younger.
  3. 9 years old.
  4. 10 years old.
  5. 11 years old.
  6. 12 years old.
  7. 13 years old.
  8. 14 years old.
  9. 15 years old or later.

CR12: About how many cigarettes have you smoked in your entire life?
  1: I have never smoked cigarettes, not even one or two puffs.
  2: 1 or more puffs but never a whole cigarette
  3: 1 cigarette
  4: 2 to 5 cigarettes
  5: 6 to 15 cigarettes (about 1/2 a pack total)
  6: 16 to 25 cigarettes (about 1 pack total)
  7: 26 to 99 cigarettes (more than 1 pack,  but less than 5 packs)
  8: 100 or more cigarettes (5 or more packs)

CR13: During the past 30 days, on how many days did you smoke cigarettes?
  1. 0 days
  2. 1-2 days
  3. 3-5 days
  4. 6-9 days
  5. 10-19 days
  6. 20 to 29 days
  7. all 30 days

CR54: Do you want to stop smoking cigarettes for good?
  1. I do not smoke now
  2. Yes
  3. No

CR56: During the past 12 months, how many times have you stopped smoking for one day or longer because you were trying to quit smoking cigarettes for good?
  1. I did not smoke during the past 12 months
  2. I did not try to quit during the past 12 months
  3. 1 time
  4. 2 times
  5. 3 to 5 times
  6. 6 to 9 times
  7. 10 or more times

Cigars:

CR21: Have you ever tried smoking cigars, cigarillos, or little cigars, even one or two puffs?
  1. yes
  2. no

CR22:How old were you when you first tried smoking a cigar, cigarillo, or little cigar, even one or two puffs?
  1. I have never smoked cigars, cigarillos, or little cigars, not even one or two puffs
  2. 8 years old or younger
  3. 9 years old
  4. 10 years old
  5. 11 years old
  6. 12 years old
  7. 13 years old
  8. 14 years old
  9. 15 years old
  10. 16 years old
  11. 17 years old  
  12. 18 years old
  13. 19 years old or older

CR23: During the past 30 days, on how many days did you smoke cigars, cigarillos, or little cigars?
  1. 0 days
  2. 1-2 days
  3. 3-5 days
  4. 6-9 days
  5. 10-19 days
  6. 20 to 29 days
  7. all 30 days

**9. Processed Raw data file**

The raw data file is alabama16_6rigions_ms.sas7bdat.cvs, Alabama14_ms.xlsx, Alabama12_ms.xlsx, which are on the same drive.

Combining Columns CR4, CR5A to CR5E to merge it to CR5:
If a row has a value in "CR5A", set the value in "CR5" to 1. If a row has a value in "CR5B", set the value in "CR5'' to 2. Because CR5A is Indian based on the file Alabama YTS 2016-MS codebook,it corresponds to the value 1, which means Indian on our race map.

Similarly, "CR5C" corresponds to 3, "CR5D" to 4, and "CR5E" to 5.

CR4 is a survey question ask the person whether or not is Hispanic. 1 is no. 2,3,4,5 each means specifc Hispanic group. I will set the value in my "CR5" to 6, which means Hispanic on my map.

If a row has values in more than one of the columns "CR5A" to "CR5E" or in CR4, set the value in "CR5" to 7, which means two races or more on our map. My map is race_mapping = {'Indian': 1, 'Asian': 2, 'Black': 3, 'Hawaii': 4, 'White': 5, 'Hispanic':6, 'Two race': 7}

Selecting Specific Columns:

From the filtered rows, select the following columns for gender, grade and race:  "CR2", "CR3", "CR5".

select the following columns for the survey questions: "CR7", "CR11", "CR12", "CR13", "CR21", "CR22", "CR23", "CR54", and "CR56".


I store the filter result into data_2012, data_2014, data_2016.

In [ ]:
import pandas as pd
def merge_cr5(row):
    cr5_cols = ['CR5A', 'CR5B', 'CR5C', 'CR5D', 'CR5E']
    cr5_flags = [1, 2, 3, 4, 5]
    cr5_values = [not pd.isna(row[col]) for col in cr5_cols]  # generates a Boolean list indicating whether each column has a value

    if sum(cr5_values) > 1:  # 7 represents two or more races
        return 7
    elif row['CR4'] in [2, 3, 4, 5] and any(cr5_values):
        return 7
    elif sum(cr5_values) == 1:
        return cr5_flags[cr5_values.index(True)]
    elif row['CR4'] in [2, 3, 4, 5]:
        return 6
    else:
        return None

# Preprocess 2012 data
data_2012 = pd.read_excel('/content/drive/My Drive/Alabama12_ms.xlsx')
data_2012 = data_2012.copy()
data_2012['CR5'] = data_2012.apply(merge_cr5, axis=1)
selected_columns = ['CR2', 'CR3', 'CR5', 'CR7', 'CR11', 'CR12', 'CR13', 'CR21', 'CR22', 'CR23', 'CR54', 'CR56']
data_2012 = data_2012.loc[:, selected_columns]

data_2014 = pd.read_excel('/content/drive/My Drive/Alabama14_ms.xlsx')
data_2014 = data_2014.copy()
data_2014['CR5'] = data_2014.apply(merge_cr5, axis=1)
data_2014 = data_2014.loc[:, selected_columns]

data_2016 = pd.read_csv('/content/drive/My Drive/alabama16_6rigions_ms.sas7bdat.csv')
data_2016 = data_2016.copy()
data_2016['CR5'] = data_2016.apply(merge_cr5, axis=1)
data_2016 = data_2016.loc[:, selected_columns]

# Load the 2024 input data
inputs_2024 = pd.read_csv('/content/drive/My Drive/2024_sample_data_not_survey.csv')
required_columns = ['CR2', 'CR3', 'CR5']
# Ensure the 2024 input data columns match the required columns
conditions_2024 = inputs_2024.loc[:, required_columns]

In [ ]:
data_2012[required_columns] = data_2012[required_columns].fillna(0)
data_2014[required_columns] = data_2014[required_columns].fillna(0)
data_2016[required_columns] = data_2016[required_columns].fillna(0)
conditions_2024.fillna(0, inplace=True)

**10. Model Selection and Training**

KNN model: a non-parametric, supervised learning classifier, which uses proximity to make classifications or predictions about the grouping of an individual data point.

Random Rorest Classifier: a meta estimator that fits a number of decision tree classifiers on various sub-samples of the dataset and uses averaging to improve the predictive accuracy and control over-fitting.

I trained the 2012, 2014, 2016 Alabama data into these models and choose the most optimal parameter in each model to predict 2024 Alabama data , but I don't include year as a feature, because 2024 year is far way from 2016 year. As I look further back in time, the distance between the vectors of the year becomes greater and greater, largely affecting accuracy. Therefore, the input is only CR2, CR3, CR5. And the output is "CR7", "CR11", "CR12", "CR13", "CR21", "CR22", "CR23", "CR54", and "CR56".

When I first attempted to train my model, I noticed that the predicted values for the 2024 dataset, such as the column CR26, were all '2'. This prompted me to review each output column in my training set, where I discovered that most values were dominated by a single number, indicating an uneven distribution. To address the underrepresented groups, I decided to use SMOTE. SMOTE (Synthetic Minority Over-sampling Technique) is an oversampling technique for binary or multiclass tasks, designed to handle class imbalance issues. It generates new synthetic samples in the feature space to increase the number of minority class samples.

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import ADASYN, RandomOverSampler

# Concatenate the datasets
train_data = pd.concat([data_2012, data_2014, data_2016], ignore_index=True)

# Shuffle the data
train_data = train_data.sample(frac=1)

# Input and output columns
input_cols = ['CR2', 'CR3', 'CR5']
output_cols = ['CR7', 'CR11', 'CR12', 'CR13', 'CR21', 'CR22', 'CR23', 'CR54', 'CR56']

# Prepare features and labels
X = train_data[input_cols]
y = train_data[output_cols]

# Split the train, validation, and test set by 0.4, 0.3, 0.3
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.4, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# Define KNN hyperparameter grid
knn_param_grid = {
    'n_neighbors': [3, 5, 7, 9],
    'weights': ['uniform', 'distance'],
    'metric': ['euclidean', 'manhattan', 'minkowski']
}

# Define RandomForest hyperparameter grid
rf_param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10]
}

# Initialize dictionary to store the best model for each column
best_models = {}

# Loop through each output column and perform SMOTE, model training, and prediction separately
for col in output_cols:
    print(f"\nTraining and evaluating models for {col}")

    # Extract the current column for training and testing
    y_train_col = y_train[col].dropna()  # Drop NaN values in the current column
    X_train_col = X_train.loc[y_train_col.index]  # Adjust X_train to match the filtered y_train_col

    y_val_col = y_val[col].dropna()
    X_val_col = X_val.loc[y_val_col.index]

    y_test_col = y_test[col].dropna()
    X_test_col = X_test.loc[y_test_col.index]

    # Standardize the feature values for the current column
    scaler = StandardScaler()
    X_train_col_scaled = scaler.fit_transform(X_train_col)
    X_val_col_scaled = scaler.transform(X_val_col)
    X_test_col_scaled = scaler.transform(X_test_col)

    min_samples = y_train_col.value_counts().min()
    class_counts = y_train_col.value_counts()
    majority_class_size = class_counts.max()

    # Adjust the number of samples for each class (keep majority unchanged, increase minority)
    sampling_strategy = {}

    for cls, count in class_counts.items():
        if count < majority_class_size:
            # Increase the number of samples for minority classes
            sampling_strategy[cls] = majority_class_size
        else:
            # Keep the majority class the same
            sampling_strategy[cls] = count
    if min_samples > 1:
        smote = SMOTE(sampling_strategy=sampling_strategy, random_state=42, k_neighbors=min(min_samples - 1, 5))
        X_smote_resampled, y_smote_resampled = smote.fit_resample(X_train_col_scaled, y_train_col)

        # Further oversample the minority class
        ros = RandomOverSampler(sampling_strategy='auto', random_state=42)
        X_train_resampled, y_train_resampled = ros.fit_resample(X_smote_resampled, y_smote_resampled)
    else:
        # Apply RandomOverSampler directly for very small classes
        ros = RandomOverSampler(sampling_strategy='auto', random_state=42)
        X_train_resampled, y_train_resampled = ros.fit_resample(X_train_col_scaled, y_train_col)

    # Use GridSearchCV to adjust KNN hyperparameter
    knn = KNeighborsClassifier()
    knn_grid_search = GridSearchCV(knn, knn_param_grid, cv=5, scoring='accuracy')
    knn_grid_search.fit(X_train_resampled, y_train_resampled)

    print(f"KNN Best Hyperparameters for {col}: {knn_grid_search.best_params_}")

    # Use the best parameters to train the KNN model
    best_knn = knn_grid_search.best_estimator_

    # Predict on the validation set
    y_val_knn_pred = best_knn.predict(X_val_col_scaled)
    val_knn_accuracy = accuracy_score(y_val_col, y_val_knn_pred)
    print(f"KNN Validation Accuracy for {col}: {val_knn_accuracy}")

    # Define RandomForest hyperparameter
    rf = RandomForestClassifier(random_state=42)
    rf_grid_search = GridSearchCV(rf, rf_param_grid, cv=5, scoring='accuracy')
    rf_grid_search.fit(X_train_resampled, y_train_resampled)

    print(f"RandomForest Best Hyperparameters for {col}: {rf_grid_search.best_params_}")

    best_rf = rf_grid_search.best_estimator_

    # Predict on the validation set
    y_val_rf_pred = best_rf.predict(X_val_col_scaled)
    val_rf_accuracy = accuracy_score(y_val_col, y_val_rf_pred)
    print(f"RandomForest Validation Accuracy for {col}: {val_rf_accuracy}")

    # Choose between RandomForest and KNN based on validation accuracy
    if val_knn_accuracy > val_rf_accuracy:
        best_model = best_knn
        print(f"Best Model for {col}: KNN")
    else:
        best_model = best_rf
        print(f"Best Model for {col}: RandomForest")

    # Save the best model for this column
    best_models[col] = best_model

    # Performance on the test set
    y_test_pred = best_model.predict(X_test_col_scaled)
    test_accuracy = accuracy_score(y_test_col, y_test_pred)
    print(f"Test Accuracy for {col}: {test_accuracy}")

# The best_models dictionary now contains the best model for each output column.
best_models


Training and evaluating models for CR7
KNN Best Hyperparameters for CR7: {'metric': 'manhattan', 'n_neighbors': 5, 'weights': 'distance'}
KNN Validation Accuracy for CR7: 0.7013574660633484
RandomForest Best Hyperparameters for CR7: {'max_depth': 10, 'min_samples_split': 5, 'n_estimators': 50}
RandomForest Validation Accuracy for CR7: 0.6119909502262444
Best Model for CR7: KNN
Test Accuracy for CR7: 0.6812080536912751

Training and evaluating models for CR11
KNN Best Hyperparameters for CR11: {'metric': 'manhattan', 'n_neighbors': 3, 'weights': 'uniform'}
KNN Validation Accuracy for CR11: 0.7954939341421143
RandomForest Best Hyperparameters for CR11: {'max_depth': 10, 'min_samples_split': 5, 'n_estimators': 100}
RandomForest Validation Accuracy for CR11: 0.231369150779896
Best Model for CR11: KNN
Test Accuracy for CR11: 0.8124459809853068

Training and evaluating models for CR12
KNN Best Hyperparameters for CR12: {'metric': 'euclidean', 'n_neighbors': 7, 'weights': 'distance'}
KNN Val

{'CR7': KNeighborsClassifier(metric='manhattan', weights='distance'),
 'CR11': KNeighborsClassifier(metric='manhattan', n_neighbors=3),
 'CR12': KNeighborsClassifier(metric='euclidean', n_neighbors=7, weights='distance'),
 'CR13': KNeighborsClassifier(metric='euclidean', n_neighbors=9),
 'CR21': KNeighborsClassifier(metric='manhattan', weights='distance'),
 'CR22': KNeighborsClassifier(metric='manhattan', n_neighbors=9, weights='distance'),
 'CR23': KNeighborsClassifier(metric='euclidean', n_neighbors=3),
 'CR54': KNeighborsClassifier(metric='euclidean', n_neighbors=3, weights='distance'),
 'CR56': KNeighborsClassifier(metric='euclidean', n_neighbors=9)}

In [ ]:
import numpy as np
predicted_df = pd.DataFrame()
conditions_2024_scaled = scaler.transform(conditions_2024)
# Loop through each output column and predict individually
for col in output_cols:

    # Get the best model for this column
    best_model = best_models[col]

    # Predict the column using the 2024 scaled input data
    predictions = best_model.predict(conditions_2024_scaled)

    # Add the predictions to the DataFrame
    predicted_df[col] = predictions


In [ ]:
result = pd.concat([inputs_2024.reset_index(drop=True), predicted_df], axis=1)
result.to_csv('classification_2024_data.csv', index=False)